In [1]:
# Import dependencies
import pandas as pd


In [2]:
# Load the data from Resources folder
df = pd.read_csv('../Resources/final_df.csv')
df.head()


,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY,BRONX,(null),(null),(null),45-64,UNKNOWN,F
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY,BRONX,UNKNOWN,UNKNOWN,U,45-64,WHITE,M
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),25-44,WHITE,M
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),45-64,UNKNOWN,M
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),65+,WHITE,F


In [3]:
crime_df = df
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6230872 entries, 0 to 6230871
Data columns (total 10 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   cmplnt_fr_dt    object
 1   cmplnt_fr_tm    object
 2   ofns_desc       object
 3   boro_nm         object
 4   susp_age_group  object
 5   susp_race       object
 6   susp_sex        object
 7   vic_age_group   object
 8   vic_race        object
 9   vic_sex         object
dtypes: object(10)
memory usage: 475.4+ MB


In [5]:
# Code update to correctly parse datetimes with milliseconds
crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'].str.replace('T', ' ') + ' ' + crime_df['cmplnt_fr_tm'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

# Checking if the conversion is successful
print(crime_df['datetime'].head())


0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: datetime, dtype: datetime64[ns]


In [6]:
crime_df.head()

,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,datetime
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY,BRONX,(null),(null),(null),45-64,UNKNOWN,F,NaT
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY,BRONX,UNKNOWN,UNKNOWN,U,45-64,WHITE,M,NaT
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),25-44,WHITE,M,NaT
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),45-64,UNKNOWN,M,NaT
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),65+,WHITE,F,NaT


In [8]:
# How many Nat values in datetime column
crime_df['datetime'].isna().sum()

6230872

In [9]:
# Filtering rows where the time is marked as '(null)' or is not in the expected format
invalid_time_rows = crime_df[crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]

# Showing the number of such rows
print(f"Number of rows with time marked as '(null)': {invalid_time_rows.shape[0]}")

# Showing those rows
print(invalid_time_rows[['cmplnt_fr_dt', 'cmplnt_fr_tm']])


C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\2225513793.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  invalid_time_rows = crime_df[crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]


Number of rows with time marked as '(null)': 4
                    cmplnt_fr_dt cmplnt_fr_tm
2175985  2013-05-08T00:00:00.000       (null)
5155950  2010-02-19T00:00:00.000       (null)
5653866  2011-08-12T00:00:00.000       (null)
5779185  2013-06-22T00:00:00.000       (null)


In [10]:
# Deleting rows with invalid time from crime_df
crime_df = crime_df[~crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]

C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\2277994098.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  crime_df = crime_df[~crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]


In [11]:
# Warning: This operation will last for a few minutes due to the large size of the dataset

# First, we merge the columns 'cmplnt_fr_dt' and 'cmplnt_fr_tm' into one column
crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'] + ' ' + crime_df['cmplnt_fr_tm'])

# Renaming the 'datetime' column to 'ds' for use in Prophet
crime_df.rename(columns={'datetime': 'ds'}, inplace=True)



C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\208940808.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'] + ' ' + crime_df['cmplnt_fr_tm'])
C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\208940808.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'] + ' ' + crime_df['cmplnt_fr_tm'])
C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\208940808.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [12]:
display(crime_df.head())
display(crime_df.tail())

,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,ds
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY,BRONX,(null),(null),(null),45-64,UNKNOWN,F,2010-10-27 19:30:00
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY,BRONX,UNKNOWN,UNKNOWN,U,45-64,WHITE,M,2021-05-24 03:00:00
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),25-44,WHITE,M,2010-02-10 12:00:00
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),45-64,UNKNOWN,M,2010-02-09 15:00:00
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),65+,WHITE,F,2010-07-29 08:00:00


,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,ds
6230867,2022-09-22T00:00:00.000,16:07:00,ESCAPE 3,QUEENS,25-44,BLACK,M,UNKNOWN,UNKNOWN,E,2022-09-22 16:07:00
6230868,2022-08-07T00:00:00.000,00:51:00,ESCAPE 3,BROOKLYN,25-44,BLACK,M,UNKNOWN,UNKNOWN,E,2022-08-07 00:51:00
6230869,2022-12-19T00:00:00.000,06:36:00,ESCAPE 3,QUEENS,25-44,WHITE HISPANIC,F,UNKNOWN,UNKNOWN,D,2022-12-19 06:36:00
6230870,2022-11-26T00:00:00.000,14:15:00,ESCAPE 3,QUEENS,<18,BLACK,M,<18,BLACK,M,2022-11-26 14:15:00
6230871,2022-05-25T00:00:00.000,22:15:00,ESCAPE 3,QUEENS,<18,BLACK,M,UNKNOWN,UNKNOWN,E,2022-05-25 22:15:00


In [13]:
# delete cmplnt_fr_dt and cmplnt_fr_tm columns
crime_df.drop(columns=['cmplnt_fr_dt', 'cmplnt_fr_tm'], inplace=True)
display(crime_df.head())
display(crime_df.tail())

C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\3096340551.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df.drop(columns=['cmplnt_fr_dt', 'cmplnt_fr_tm'], inplace=True)


,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,ds
0,PETIT LARCENY,BRONX,(null),(null),(null),45-64,UNKNOWN,F,2010-10-27 19:30:00
1,PETIT LARCENY,BRONX,UNKNOWN,UNKNOWN,U,45-64,WHITE,M,2021-05-24 03:00:00
2,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),25-44,WHITE,M,2010-02-10 12:00:00
3,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),45-64,UNKNOWN,M,2010-02-09 15:00:00
4,PETIT LARCENY,STATEN ISLAND,(null),(null),(null),65+,WHITE,F,2010-07-29 08:00:00


,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex,ds
6230867,ESCAPE 3,QUEENS,25-44,BLACK,M,UNKNOWN,UNKNOWN,E,2022-09-22 16:07:00
6230868,ESCAPE 3,BROOKLYN,25-44,BLACK,M,UNKNOWN,UNKNOWN,E,2022-08-07 00:51:00
6230869,ESCAPE 3,QUEENS,25-44,WHITE HISPANIC,F,UNKNOWN,UNKNOWN,D,2022-12-19 06:36:00
6230870,ESCAPE 3,QUEENS,<18,BLACK,M,<18,BLACK,M,2022-11-26 14:15:00
6230871,ESCAPE 3,QUEENS,<18,BLACK,M,UNKNOWN,UNKNOWN,E,2022-05-25 22:15:00


In [14]:
# make ds column the index and sort the index in ascending order
crime_df.set_index('ds', inplace=True)
crime_df.sort_index(inplace=True)

display(crime_df.head())
display(crime_df.tail())

C:\Users\abeli\AppData\Local\Temp\ipykernel_19700\1125920588.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_df.sort_index(inplace=True)


,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
ds,,,,,,,,
2010-01-01,RAPE,BROOKLYN,25-44,WHITE HISPANIC,M,<18,WHITE HISPANIC,F
2010-01-01,SEX CRIMES,BROOKLYN,18-24,UNKNOWN,F,<18,UNKNOWN,F
2010-01-01,OFF. AGNST PUB ORD SENSBLTY &,BRONX,25-44,WHITE HISPANIC,F,18-24,WHITE HISPANIC,F
2010-01-01,SEX CRIMES,BRONX,UNKNOWN,WHITE HISPANIC,M,<18,WHITE HISPANIC,F
2010-01-01,CRIMINAL MISCHIEF & RELATED OF,BROOKLYN,(null),(null),(null),(null),UNKNOWN,D


,ofns_desc,boro_nm,susp_age_group,susp_race,susp_sex,vic_age_group,vic_race,vic_sex
ds,,,,,,,,
2022-12-31 23:30:00,SEX CRIMES,MANHATTAN,25-44,WHITE HISPANIC,M,18-24,WHITE,F
2022-12-31 23:30:00,HARRASSMENT 2,BROOKLYN,25-44,BLACK,M,18-24,BLACK,F
2022-12-31 23:40:00,INTOXICATED & IMPAIRED DRIVING,BRONX,25-44,WHITE HISPANIC,M,UNKNOWN,UNKNOWN,E
2022-12-31 23:50:00,GRAND LARCENY,MANHATTAN,UNKNOWN,UNKNOWN,U,18-24,WHITE,F
2022-12-31 23:55:00,CRIMINAL MISCHIEF & RELATED OF,MANHATTAN,18-24,UNKNOWN,F,UNKNOWN,UNKNOWN,D


In [17]:
crime_df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6230868 entries, 2010-01-01 00:00:00 to 2022-12-31 23:55:00
Data columns (total 8 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   ofns_desc       object
 1   boro_nm         object
 2   susp_age_group  object
 3   susp_race       object
 4   susp_sex        object
 5   vic_age_group   object
 6   vic_race        object
 7   vic_sex         object
dtypes: object(8)
memory usage: 427.8+ MB


In [15]:
# Save the cleaned data to a new csv file
crime_df.to_csv('../Resources/cleaned_crime_data.csv')
